<a href="https://colab.research.google.com/github/Debottam/tensorFlowCustomized/blob/master/customizedTensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>